# Data Extraction from API

In [5]:
import pandas as pd

## BKK Policy Tracking Webscrapping

In [86]:
bkk_url = 'https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findex'

# Read HTML tables into a list of DataFrame objects.
all_tables = pd.read_html(bkk_url)
# Check number of table(s)
len(all_tables)

1

In [87]:
df_rushing_policy = all_tables[0]
df_rushing_policy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   เป้าหมายตามนโยบาย ผว.กทม.  30 non-null     object 
 1   หน่วยนับ                   30 non-null     object 
 2   เป้าหมายรวม                30 non-null     float64
 3   เป้าหมายจากสำนัก           30 non-null     float64
 4   เป้าหมายจาก 50 เขต         30 non-null     float64
 5   ผลการดำเนินงาน*            30 non-null     float64
 6   ความคืบหน้า* ของ KPI (%)   31 non-null     float64
dtypes: float64(5), object(2)
memory usage: 1.8+ KB


In [8]:
pd.set_option('display.float_format', str)
pd.set_option('display.max_colwidth', None)

In [88]:
df_rushing_policy.columns

Index(['เป้าหมายตามนโยบาย ผว.กทม.', 'หน่วยนับ', 'เป้าหมายรวม',
       'เป้าหมายจากสำนัก', 'เป้าหมายจาก 50 เขต', 'ผลการดำเนินงาน*',
       'ความคืบหน้า* ของ KPI (%)'],
      dtype='object')

In [89]:
df_rushing_policy.dropna(axis = 0, inplace = True)

new_name = {'เป้าหมายตามนโยบาย ผว.กทม.': 'Goal', 'หน่วยนับ':'Unit', 'เป้าหมายรวม': 'Total Goals',
       'เป้าหมายจากสำนัก':'Goal (Departments)', 'เป้าหมายจาก 50 เขต':'Goal (50 Districts)', 'ผลการดำเนินงาน*':'Total Progress (Unit)',
       'ความคืบหน้า* ของ KPI (%)':'Progress (%)'}

df_rushing_policy.rename(columns = new_name, inplace = True)

# new data frame with split value columns
new = df_rushing_policy["Goal"].str.split(":", n=1, expand=True)

# making separate first name column from new data frame
df_rushing_policy["Goal"] = new[0]

# making separate last name column from new data frame
df_rushing_policy["Related OKR"] = new[1]

# Dropping old Name columns
# df_rushing_policy.drop(columns=["Goal"], inplace=True)

df_rushing_policy['Related OKR'] = df_rushing_policy['Related OKR'].str.replace('OKR ', '')
df_rushing_policy['Related OKR'] = df_rushing_policy['Related OKR'].str.replace('(** ค่าเฉลี่ย **)', '(Mean Value)')

new_2 = df_rushing_policy["Goal"].str.split(" ", n=1, expand=True)
df_rushing_policy["No. (Goal)"] = new_2[0]
df_rushing_policy["Goal"] = new_2[1]

df_rushing_policy = df_rushing_policy[['No. (Goal)', 'Related OKR', 'Goal', 'Unit', 'Total Goals', 'Goal (Departments)',
       'Goal (50 Districts)', 'Total Progress (Unit)', 'Progress (%)']]

df_rushing_policy['No. (Goal)'] = df_rushing_policy['No. (Goal)'].str.strip('.')

In [90]:
df_rushing_policy.head(10)

,No. (Goal),Related OKR,Goal,Unit,Total Goals,Goal (Departments),Goal (50 Districts),Total Progress (Unit),Progress (%)
0,1,4.1.10,พัฒนาถนนสวย (กม.),กม.,133.35,0.0,133.35,126.26,94.69
1,2,4.1.01,ปลูกต้นไม้รวมทุกประเภท (ต้น),ต้น,200000.0,50000.0,150000.0,302777.0,151.39
2,3,4.1.05,เพิ่มสวน 15 นาที (แห่ง),แห่ง,156.0,6.0,150.0,126.0,80.77
3,4.1,1.1.05,ปรับปรุงทางเท้า (กม.),กม.,324.11,300.0,24.11,109.39,33.75
4,4.2,1.3.07,ทางจักรยาน (กม.),กม.,518.0,18.0,500.0,223.87,43.22
5,4.3,None,ปรับปรุงซอย (กม.),กม.,136.0,0.0,136.0,75.82,55.76
6,5,2.1.20,ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง),ดวง,32078.0,13100.0,18978.0,40355.0,125.8
7,6,1.1.03,ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง),ดวง,8056.0,2114.0,5942.0,4706.28,58.42
8,7,1.1.01,ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อนผัน (แห่ง),แห่ง,109.0,0.0,109.0,114.0,104.59
9,8,7.2.05,จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center รองรับผู้ค้าหาบเร่ (แห่ง),แห่ง,28.0,0.0,28.0,26.0,92.85


In [84]:
df_rushing_policy.to_csv('df_rushing_policies.csv')

## BKK: Monthly Progress of Policy

In [11]:
import pandas as pd
import numpy as np
import re

### Initialized DataFrame

In [58]:
# Declare Blank DataFrame
df_progress = pd.DataFrame()

# Access and concat all DataFrames to same DataFrames
for i in range(1,7):
    policy_url = "https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=" + str(i) +"&per-page=50"
    progress_table = pd.read_html(policy_url)
    df_progress = pd.concat([df_progress, progress_table[0]], ignore_index=True, axis = 0)

## Rename Columns
new_col_names = {'KEY_RESULT':'Goal', 'ค่าเป้าหมาย/ปี':'Yearly Goal', 'ผลดำเนินงาน (รวม)':'Total Progress (Unit)', 'หน่วยนับ':'Unit',
       'ตค.66':'Oct 23', 'พย.66':'Nov 23', 'ธค.66':'Dec 23', 'มค.67':'Jan 24', 'กพ.67':'Feb 24', 'มี.ค.67':'Mar 24', 'เม.ย.67':'Apr 24',
       'พค.67':'May 24', 'มิ.ย.67':'Jun 24', 'กค.67':'July 24', 'สค.67':'Aug 24', 'กย.67':'Sept 24'}
df_progress.rename(columns = new_col_names, inplace = True)

# Drop Null Value
df_progress.dropna(axis = 0, inplace = True)

# Reset Index
df_progress = df_progress.reset_index()
df_progress.drop('index', axis = 1, inplace = True)

# Set Index
df_progress['#'] = df_progress['#'].astype('int')
df_progress.set_index(df_progress['#'], inplace = True)
df_progress.drop('#', axis = 1, inplace = True)

In [ ]:
df_progress.head()

,Goal,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
#,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 105.05%,123.6,129.84,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,22.51,0.0,0.0
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 149.45%,200000.0,298905.0,ต้น,8734.0,24454.0,14886.0,29427.0,38556.0,51876.0,29415.0,25710.0,35950.0,39897.0,0.0,0.0
3,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 79.49%,156.0,124.0,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,24.0,0.0,0.0
4,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.44%,312.07,113.73,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.89,0.0,0.0
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1.20 125.81%,31878.0,40105.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,3418.0,0.0,0.0


### Extract OKR value

In [59]:
# Function to extract all occurrences of "OKR \d+\.\d+\.\d+" (OKR 0.0.0)
def extract_okr_numbers(text):
    okrs = re.findall(r'OKR \d+\.\d+\.\d+(?:\.\d+)?', text)
    okrs = ''.join(okrs)
    result = okrs.replace('OKR ', '', )
    return result

def remove_okr_numbers(text):
    return re.sub(r'OKR \d+\.\d+\.\d+(?:\.\d+)?', '', text).strip()

# Apply the function to the Goal column
df_progress['Related OKRs'] = df_progress['Goal'].apply(extract_okr_numbers)
df_progress['Goal'] = df_progress['Goal'].apply(lambda x: remove_okr_numbers(x))


# Replace all Blank value by NaN (Null)
df_progress['Related OKRs'] = df_progress['Related OKRs'].replace('', np.nan)

### Extract KPI value

In [60]:
# Function to extract all occurrences of "\d+\.\d+%" (xxx.xx%)
def extract_kpi_numbers(text):
    kpi = re.findall(r'KPI \d+(?:\.\d+){2,3}', text)
    kpi = ''.join(kpi)
    result = kpi.replace('KPI ', '', )
    return result

def remove_kpi_numbers(text):
    return re.sub(r'KPI \d+(?:\.\d+){2,3}', '', text).strip()

df_progress['Related KPI'] = df_progress['Goal'].apply(extract_kpi_numbers)
df_progress['Goal'] = df_progress['Goal'].apply(lambda x: remove_kpi_numbers(x))

df_progress['Related KPI'] = df_progress['Related KPI'].replace('', np.nan)

### Extract Percent Progress

In [61]:
# Function to extract all occurrences of "\d+\.\d+%" (xxx.xx%)
def extract_percent_progress(text):
    progress_percent = re.findall(r'\d{1,3}(?:,\d{3})*\.\d+%', text)
    return ''.join(progress_percent)

def remove_percent_progress(text):
    return re.sub(r'\d{1,3}(?:,\d{3})*\.\d+%', '', text).strip()


# Dealing with Total Progress (%)
df_progress['Total Progress (%)'] = df_progress['Goal'].apply(extract_percent_progress)
df_progress['Goal'] = df_progress['Goal'].apply(lambda x: remove_percent_progress(x))

df_progress['Goal'] = df_progress['Goal'].str.replace(':', '')

#Check DF info
df_progress.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 1 to 300
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Goal                   300 non-null    object 
 1   Yearly Goal            300 non-null    float64
 2   Total Progress (Unit)  300 non-null    float64
 3   Unit                   300 non-null    object 
 4   Oct 23                 300 non-null    float64
 5   Nov 23                 300 non-null    float64
 6   Dec 23                 300 non-null    float64
 7   Jan 24                 300 non-null    float64
 8   Feb 24                 300 non-null    float64
 9   Mar 24                 300 non-null    float64
 10  Apr 24                 300 non-null    float64
 11  May 24                 300 non-null    float64
 12  Jun 24                 300 non-null    float64
 13  July 24                300 non-null    float64
 14  Aug 24                 300 non-null    float64
 15  Sept 24    

### Extract Goal_ID

In [80]:
def extract_goal_id(text):
    goal_id = re.findall(r'\d+\.\d?', str(text))
    result = ''.join(goal_id)
    return result
# def extract_patterns(text):
#     return re.findall(r'\b\d+\.\d*\b', text)

df_progress['No. (Goal)'] = df_progress['Goal'].apply(extract_goal_id)
df_progress['No. (Goal)'] = df_progress['No. (Goal)'].replace('', np.nan)
df_progress['No. (Goal)'] = df_progress['No. (Goal)'].str.strip('.')

In [81]:
df_progress = df_progress[['Goal','No. (Goal)','Unit', 'Related OKRs', 'Related KPI', 'Yearly Goal', 'Total Progress (Unit)', 'Total Progress (%)',
             'Oct 23', 'Nov 23', 'Dec 23', 'Jan 24', 'Feb 24', 'Mar 24', 'Apr 24', 'May 24', 'Jun 24', 'July 24', 'Aug 24', 'Sept 24']]

In [82]:
df_progress.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 1 to 300
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Goal                   300 non-null    object 
 1   No. (Goal)             27 non-null     object 
 2   Unit                   300 non-null    object 
 3   Related OKRs           58 non-null     object 
 4   Related KPI            254 non-null    object 
 5   Yearly Goal            300 non-null    float64
 6   Total Progress (Unit)  300 non-null    float64
 7   Total Progress (%)     300 non-null    object 
 8   Oct 23                 300 non-null    float64
 9   Nov 23                 300 non-null    float64
 10  Dec 23                 300 non-null    float64
 11  Jan 24                 300 non-null    float64
 12  Feb 24                 300 non-null    float64
 13  Mar 24                 300 non-null    float64
 14  Apr 24                 300 non-null    float64
 15  May 24     

In [79]:
df_progress.head(40)

,Goal,Goal_ID,Unit,Related OKRs,Related KPI,Yearly Goal,Total Progress (Unit),Total Progress (%),Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
#,,,,,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.),1.,กม.,4.1.10,NaN,123.6,126.85,102.63%,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,9.15,18.02,0.0,0.0
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น),2.,ต้น,4.1.01,NaN,200000.0,301839.0,150.92%,8734.0,24454.0,14792.0,29427.0,38556.0,51876.0,29474.0,25720.0,36038.0,39084.0,3684.0,0.0
3,3. เพิ่มสวน 15 นาที (แห่ง),3.,แห่ง,4.1.05,NaN,156.0,128.0,82.05%,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,20.0,20.0,2.0,0.0
4,4.1 ปรับปรุงทางเท้า (กม.),4.1,กม.,1.1.05,NaN,312.07,114.19,36.59%,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.73,94.09,0.16,0.0
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง),5.,ดวง,2.1.20,NaN,31878.0,40661.0,127.55%,510.0,3578.0,3860.0,5048.0,3860.0,6707.0,3924.0,4371.0,3630.0,4732.0,441.0,0.0
6,6. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง),6.,ดวง,1.1.03,NaN,9536.0,4721.28,49.51%,70.0,316.0,234.0,325.0,1413.0,345.0,507.0,472.0,416.0,623.28,0.0,0.0
7,7. ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อนผัน (แห่ง),7.,แห่ง,1.1.01,NaN,100.0,115.0,115.00%,3.0,1.0,1.0,12.0,14.0,30.0,23.0,12.0,7.0,11.0,1.0,0.0
8,8. จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center รองรับผู้ค้าหาบเร่ (แห่ง),8.,แห่ง,7.2.05,NaN,11.0,30.0,272.73%,1.0,3.0,1.0,4.0,2.0,7.0,4.0,4.0,1.0,2.0,1.0,0.0
9,9. แก้ไขจุดเสี่ยงน้ำท่วม (จุด),9.,จุด,1.4.05,NaN,212.0,223.0,105.19%,5.0,22.0,1.0,5.0,8.01,32.0,12.2,17.2,7.45,113.14,0.0,0.0


In [85]:
df_progress.to_csv('all_progress.csv')

# Joining df_rushing_policy to df_progress

In [113]:
df_joined = df_rushing_policy.merge(df_progress, left_on = 'No. (Goal)', right_on = 'No. (Goal)', how = 'left',suffixes=('_Rush', '_All'))

In [115]:
df_joined = df_joined[['No. (Goal)','Goal_All', 'Related OKR', 'Total Progress (Unit)_All', 'Total Progress (%)', 'Oct 23', 'Nov 23',
       'Dec 23', 'Jan 24', 'Feb 24', 'Mar 24', 'Apr 24', 'May 24', 'Jun 24',
       'July 24', 'Aug 24', 'Sept 24']]

In [103]:
df_joined.columns

Index(['No. (Goal)', 'Related OKR', 'Goal_Rush', 'Unit_Rush', 'Total Goals',
       'Goal (Departments)', 'Goal (50 Districts)',
       'Total Progress (Unit)_Rush', 'Progress (%)', 'Goal_All', 'Unit_All',
       'Related OKRs', 'Related KPI', 'Yearly Goal',
       'Total Progress (Unit)_All', 'Total Progress (%)', 'Oct 23', 'Nov 23',
       'Dec 23', 'Jan 24', 'Feb 24', 'Mar 24', 'Apr 24', 'May 24', 'Jun 24',
       'July 24', 'Aug 24', 'Sept 24'],
      dtype='object')

In [116]:
df_joined.loc[df_joined['Feb 24'].isna()]

,No. (Goal),Goal_All,Related OKR,Total Progress (Unit)_All,Total Progress (%),Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
4,4.2,NaN,1.3.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.3,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,10.2,NaN,1.2.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,11.3,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,16.3,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,23,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
df_joined.head(30)

,No. (Goal),Goal_All,Related OKR,Total Progress (Unit)_All,Total Progress (%),Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
0,1,1. พัฒนาถนนสวย (กม.),4.1.10,126.85,102.63%,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,9.15,18.02,0.0,0.0
1,2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น),4.1.01,301839.0,150.92%,8734.0,24454.0,14792.0,29427.0,38556.0,51876.0,29474.0,25720.0,36038.0,39084.0,3684.0,0.0
2,3,3. เพิ่มสวน 15 นาที (แห่ง),4.1.05,128.0,82.05%,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,20.0,20.0,2.0,0.0
3,4.1,4.1 ปรับปรุงทางเท้า (กม.),1.1.05,114.19,36.59%,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.73,94.09,0.16,0.0
4,4.2,NaN,1.3.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.3,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง),2.1.20,40661.0,127.55%,510.0,3578.0,3860.0,5048.0,3860.0,6707.0,3924.0,4371.0,3630.0,4732.0,441.0,0.0
7,6,6. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง),1.1.03,4721.28,49.51%,70.0,316.0,234.0,325.0,1413.0,345.0,507.0,472.0,416.0,623.28,0.0,0.0
8,7,7. ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อนผัน (แห่ง),1.1.01,115.0,115.00%,3.0,1.0,1.0,12.0,14.0,30.0,23.0,12.0,7.0,11.0,1.0,0.0
9,8,8. จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center รองรับผู้ค้าหาบเร่ (แห่ง),7.2.05,30.0,272.73%,1.0,3.0,1.0,4.0,2.0,7.0,4.0,4.0,1.0,2.0,1.0,0.0


In [ ]:
df_progress.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 1 to 300
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Goal                   300 non-null    object 
 1   Yearly Goal            300 non-null    float64
 2   Total Progress (Unit)  300 non-null    float64
 3   Related OKRs           58 non-null     object 
 4   Related KPI            254 non-null    object 
 5   Total Progress (%)     300 non-null    object 
 6   Unit                   300 non-null    object 
 7   Oct 23                 300 non-null    float64
 8   Nov 23                 300 non-null    float64
 9   Dec 23                 300 non-null    float64
 10  Jan 24                 300 non-null    float64
 11  Feb 24                 300 non-null    float64
 12  Mar 24                 300 non-null    float64
 13  Apr 24                 300 non-null    float64
 14  May 24                 300 non-null    float64
 15  Jun 24     

In [83]:
df_progress.head(50)

,Goal,No. (Goal),Unit,Related OKRs,Related KPI,Yearly Goal,Total Progress (Unit),Total Progress (%),Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
#,,,,,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.),1,กม.,4.1.10,NaN,123.6,126.85,102.63%,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,9.15,18.02,0.0,0.0
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น),2,ต้น,4.1.01,NaN,200000.0,301839.0,150.92%,8734.0,24454.0,14792.0,29427.0,38556.0,51876.0,29474.0,25720.0,36038.0,39084.0,3684.0,0.0
3,3. เพิ่มสวน 15 นาที (แห่ง),3,แห่ง,4.1.05,NaN,156.0,128.0,82.05%,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,20.0,20.0,2.0,0.0
4,4.1 ปรับปรุงทางเท้า (กม.),4.1,กม.,1.1.05,NaN,312.07,114.19,36.59%,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.73,94.09,0.16,0.0
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง),5,ดวง,2.1.20,NaN,31878.0,40661.0,127.55%,510.0,3578.0,3860.0,5048.0,3860.0,6707.0,3924.0,4371.0,3630.0,4732.0,441.0,0.0
6,6. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง),6,ดวง,1.1.03,NaN,9536.0,4721.28,49.51%,70.0,316.0,234.0,325.0,1413.0,345.0,507.0,472.0,416.0,623.28,0.0,0.0
7,7. ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อนผัน (แห่ง),7,แห่ง,1.1.01,NaN,100.0,115.0,115.00%,3.0,1.0,1.0,12.0,14.0,30.0,23.0,12.0,7.0,11.0,1.0,0.0
8,8. จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center รองรับผู้ค้าหาบเร่ (แห่ง),8,แห่ง,7.2.05,NaN,11.0,30.0,272.73%,1.0,3.0,1.0,4.0,2.0,7.0,4.0,4.0,1.0,2.0,1.0,0.0
9,9. แก้ไขจุดเสี่ยงน้ำท่วม (จุด),9,จุด,1.4.05,NaN,212.0,223.0,105.19%,5.0,22.0,1.0,5.0,8.01,32.0,12.2,17.2,7.45,113.14,0.0,0.0


In [ ]:
df_progress.head(40)

,Goal,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24,Related OKRs,Total Progress (%),Related KPI
#,,,,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.),123.6,129.84,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,22.51,0.0,0.0,4.1.10,105.05%,NaN
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น),200000.0,298905.0,ต้น,8734.0,24454.0,14886.0,29427.0,38556.0,51876.0,29415.0,25710.0,35950.0,39897.0,0.0,0.0,4.1.01,149.45%,NaN
3,3. เพิ่มสวน 15 นาที (แห่ง),156.0,124.0,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,24.0,0.0,0.0,4.1.05,79.49%,NaN
4,4.1 ปรับปรุงทางเท้า (กม.),312.07,113.73,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.89,0.0,0.0,1.1.05,36.44%,NaN
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง),31878.0,40105.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,3418.0,0.0,0.0,2.1.20,125.81%,NaN
6,6. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง),9536.0,4701.28,ดวง,70.0,316.0,234.0,325.0,1413.0,345.0,507.0,472.0,416.0,603.28,0.0,0.0,1.1.03,49.30%,NaN
7,7. ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อนผัน (แห่ง),100.0,114.0,แห่ง,3.0,1.0,1.0,12.0,14.0,30.0,23.0,12.0,7.0,11.0,0.0,0.0,1.1.01,114.00%,NaN
8,8. จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center รองรับผู้ค้าหาบเร่ (แห่ง),11.0,29.0,แห่ง,1.0,3.0,1.0,4.0,2.0,7.0,4.0,4.0,1.0,2.0,0.0,0.0,7.2.05,263.64%,NaN
9,9. แก้ไขจุดเสี่ยงน้ำท่วม (จุด),212.0,222.0,จุด,5.0,22.0,1.0,5.0,8.01,32.0,12.2,17.2,7.45,112.14,0.0,0.0,1.4.05,104.72%,NaN


In [ ]:
df_progress.loc[df_progress['Related KPI'].isna() == False]

,Goal,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24,Related OKRs,Related KPI,Total Progress (%)
#,,,,,,,,,,,,,,,,,,,
26,ความสำเร็จในการแก้ไขปัญหาข้อร้องเรียนของประชาชนผ่าน Traffy Fondue,90.0,17243.72,ร้อยละ,591.76,1484.03,1723.35,1918.12,2073.35,1868.41,1934.12,2483.8,1406.42,1760.36,0.0,0.0,NaN,.1,"19,159.69%"
27,เรื่องที่ได้รับการแก้ไข ปรับปรุง หรือส่งต่อหน่วยงานเจ้าของภารกิจ,100.0,142.0,ร้อยละ,10.0,0.0,0.0,0.0,0.0,0.0,50.0,10.0,72.0,0.0,0.0,0.0,NaN,.1,142.00%
28,จำนวนบริการหรือกิจกรรมสาธารณะของ กรุงเทพมหานคร ที่ประชาชนหรือภาคส่วนอื่น ๆ เข้ามามีส่วนร่วม,1.0,109.7,บริการต่อกิจกรรมสาธารณะ,0.0,0.0,0.0,15.0,94.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,.3,"10,970.00%"
29,สำนักงานเขตจัดทำแผนพัฒนาเขตที่มาจากการมีส่วนร่วมของหน่วยนโยบาย ชุมชน และภาคประชาชน และให้กรุงเทพมหานครนำแผนส่วนนี้เข้ามาพิจารณาเพื่อจัดสรรงบประมาณให้กับสำนักงานเขต,50.0,0.0,โครงการ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,.1,0.00%
30,"โครงการที่มาจากกระบวนการมีส่วนร่วมของประชาชนได้รับการบรรจุ ในแผน 2,017 ชุมชน และดำเนินการ",2017.0,163.0,ชุมชน,0.0,9.0,9.0,8.0,14.0,85.0,9.0,8.0,11.0,10.0,0.0,0.0,NaN,.2,8.08%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,กลุ่มเปราะบางที่ได้รับการจ้างงานจากหน่วยงานในสังกัดกรุงเทพมหานคร (เช่น คนพิการ ผู้สูงอายุ คนไร้บ้าน แม่ที่ท้องไม่พร้อม ฯลฯ),100.0,2684.33,ร้อยละ,639.33,259.0,263.0,160.5,173.5,253.5,251.83,162.0,270.67,251.0,0.0,0.0,NaN,.1,"2,684.33%"
276,กลุ่มเปราะบางที่เข้ารับการฝึกอาชีพที่โรงเรียน/ศูนย์ฝึกอาชีพสังกัดกรุงเทพมหานคร และนำไปประกอบอาชีพสร้างรายได้จริง (เช่น คนพิการ ผู้สูงอายุ แรงงานนอกระบบ ฯลฯ),50.0,0.0,ร้อยละ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,.2,0.00%
277,ข้าราชการ และบุคลากรของกรุงเทพมหานครที่ได้รับการอบรม ฝึกทักษะ เพื่อทำงานร่วมกับคนกลุ่มเปราะบางกลุ่มต่าง ๆ,100.0,300.0,ร้อยละ,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,100.0,0.0,0.0,0.0,NaN,.3,300.00%


In [ ]:
df_progress

,Goal,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24,Related OKRs,Related KPI,Total Progress (%)
#,,,,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.),123.6,129.84,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,22.51,0.0,0.0,NaN,NaN,
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น),200000.0,298905.0,ต้น,8734.0,24454.0,14886.0,29427.0,38556.0,51876.0,29415.0,25710.0,35950.0,39897.0,0.0,0.0,NaN,NaN,
3,3. เพิ่มสวน 15 นาที (แห่ง),156.0,124.0,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,24.0,0.0,0.0,NaN,NaN,
4,4.1 ปรับปรุงทางเท้า (กม.),312.07,113.73,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.89,0.0,0.0,NaN,NaN,
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง),31878.0,40105.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,3418.0,0.0,0.0,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,จำนวนหลักสูตรดัดแปลงรถจักรยานยนต์เป็นเครื่องยนต์ไฟฟ้า,1.0,1.0,หลักสูตร,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,
297,"จำนวนกรงตาข่ายครอบถุงขยะที่ติดตั้งถนนสายหลัก 10,000 กรง",2250.0,268.0,กรง (สะสม),0.0,0.0,0.0,0.0,0.0,150.0,118.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,
298,ติดตั้งถังขยะ 3 ประเภทขนาด 85 ลิตร (Street Furniture) สำหรับคนเดินเท้าบน ทางเท้าถนนสายหลัก ถนนสายรอง,5000.0,0.0,ชุด,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,


In [ ]:
df_progress.isnull().sum()

,0
Key_Result,0
Yearly Goal,0
Acheived Goal,0
Unit,0
Oct 23,0
Nov 23,0
Dec 23,0
Jan 24,0
Feb 24,0
Mar 24,0


In [ ]:
# # Contain OKR
df_progress.loc[df_progress['Goal'].str.contains('OKR')]

,Key_Result,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
#,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 105.05%,1.236000e+02,1.298400e+02,กม.,2.27,0.20,3.49,17.01,20.46,26.27,17.60,12.39,7.65,22.51,0.0,0.0
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 ...,2.000000e+05,2.988750e+05,ต้น,8734.00,24454.00,14886.00,29427.00,38556.00,51876.00,29415.00,25710.00,35950.00,39867.00,0.0,0.0
3,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 79.49%,1.560000e+02,1.240000e+02,แห่ง,2.00,6.00,7.00,16.00,13.00,22.00,11.00,9.00,14.00,24.00,0.0,0.0
4,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.44%,3.120700e+02,1.137300e+02,กม.,0.00,0.00,0.00,1.40,0.75,2.98,5.77,5.31,3.63,93.89,0.0,0.0
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1...,3.187800e+04,4.005500e+04,ดวง,2103.00,3578.00,4445.00,4341.00,3876.00,6195.00,3985.00,4336.00,3828.00,3368.00,0.0,0.0
6,6. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง) : ...,9.536000e+03,4.657280e+03,ดวง,70.00,316.00,234.00,325.00,1413.00,345.00,507.00,472.00,416.00,559.28,0.0,0.0
7,7. ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อน...,1.000000e+02,1.140000e+02,แห่ง,3.00,1.00,1.00,12.00,14.00,30.00,23.00,12.00,7.00,11.00,0.0,0.0
8,8. จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center ...,1.100000e+01,2.900000e+01,แห่ง,1.00,3.00,1.00,4.00,2.00,7.00,4.00,4.00,1.00,2.00,0.0,0.0
9,9. แก้ไขจุดเสี่ยงน้ำท่วม (จุด) : OKR 1.4.05 1...,2.120000e+02,2.220000e+02,จุด,5.00,22.00,1.00,5.00,8.01,32.00,12.20,17.20,7.45,112.14,0.0,0.0


In [ ]:
# Function to extract all occurrences of "OKR \d+\.\d+\.\d+"
def extract_okr_numbers(text):
    okrs = re.findall(r'OKR (\d+\.\d+\.\d+)', text)
    return ''.join(okrs)

def remove_okr_numbers(text):
    return re.sub(r'OKR \d+\.\d+\.\d+', '', text).strip()

def extract_percent_progress(text):
    progress_percent = re.findall(r'\d+\.\d+%', text)
    return ''.join(progress_percent)

def remove_percent_progress(text):
    return re.sub(r'\d+\.\d+%', '', text).strip()

def remove_semi_colon(text):
    return re.sub(r'\d+\.\d+%', '', text).strip()


# Apply the function to the Goal column
df_progress['Extracted_OKRs'] = df_progress['Goal'].apply(extract_okr_numbers)
df_progress['Goal'] = df_progress['Goal'].apply(lambda x: remove_okr_numbers(x))

df_progress['Extracted_OKRs'] = df_progress['Extracted_OKRs'].replace(r'^\s*$', np.nan, regex=True)

df_progress['Total Progress (%)'] = df_progress['Goal'].apply(extract_percent_progress)
df_progress['Goal'] = df_progress['Goal'].apply(lambda x: remove_percent_progress(x))

df_progress['Total Progress (%)'] = df_progress['Total Progress (%)'].replace(r'^\s*$', np.nan, regex=True)

df_progress['Goal'].str.replace(':', '')

df_progress.info()

KeyError: 'Goal'

In [ ]:
df_progress.head(10)

,Goal,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24,Related OKRs,Total Progress (%),Extracted_OKRs
#,,,,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.),123.6,129.84,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,22.51,0.0,0.0,NaN,NaN,NaN
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น),200000.0,298905.0,ต้น,8734.0,24454.0,14886.0,29427.0,38556.0,51876.0,29415.0,25710.0,35950.0,39897.0,0.0,0.0,NaN,NaN,NaN
3,3. เพิ่มสวน 15 นาที (แห่ง),156.0,124.0,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,24.0,0.0,0.0,NaN,NaN,NaN
4,4.1 ปรับปรุงทางเท้า (กม.),312.07,113.73,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.89,0.0,0.0,NaN,NaN,NaN
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง),31878.0,40105.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,3418.0,0.0,0.0,NaN,NaN,NaN
6,6. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง),9536.0,4701.28,ดวง,70.0,316.0,234.0,325.0,1413.0,345.0,507.0,472.0,416.0,603.28,0.0,0.0,NaN,NaN,NaN
7,7. ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อนผัน (แห่ง),100.0,114.0,แห่ง,3.0,1.0,1.0,12.0,14.0,30.0,23.0,12.0,7.0,11.0,0.0,0.0,NaN,NaN,NaN
8,8. จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center รองรับผู้ค้าหาบเร่ (แห่ง),11.0,29.0,แห่ง,1.0,3.0,1.0,4.0,2.0,7.0,4.0,4.0,1.0,2.0,0.0,0.0,NaN,NaN,NaN
9,9. แก้ไขจุดเสี่ยงน้ำท่วม (จุด),212.0,222.0,จุด,5.0,22.0,1.0,5.0,8.01,32.0,12.2,17.2,7.45,112.14,0.0,0.0,NaN,NaN,NaN


In [ ]:
df_progress.to_csv("check.csv")

In [ ]:
df_progress[['Extracted_OKRs']].to_csv('okr_list.csv')

In [ ]:
df_progress['Extracted_OKRs'] = df_progress['Extracted_OKRs'].replace(r'^\s*$', np.nan, regex=True)

In [ ]:
df_progress[['Extracted_OKRs']].isna().sum()

,0
Extracted_OKRs,242


In [ ]:
df_progress[['Extracted_OKRs']].isna().sum()

,0
Extracted_OKRs,0


In [ ]:
(df_progress['Extracted_OKRs'].str.strip() == '').sum()

242

In [ ]:
# Contain OKR, but not Contain KPI
df_progress.loc[df_progress['Goal'].str.contains('OKR') & (df_progress['Goal'].str.contains('KPI') == False)].shape

(45, 16)

In [ ]:
df_progress

,Key_Result,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24,Extracted_OKRs
#,,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 105.05%,123.60,129.84,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.60,12.39,7.65,22.51,0.0,0.0,4.1.10
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 ...,200000.00,298875.00,ต้น,8734.00,24454.0,14886.00,29427.00,38556.00,51876.00,29415.00,25710.00,35950.00,39867.00,0.0,0.0,4.1.01
3,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 79.49%,156.00,124.00,แห่ง,2.00,6.0,7.00,16.00,13.00,22.00,11.00,9.00,14.00,24.00,0.0,0.0,4.1.05
4,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.44%,312.07,113.73,กม.,0.00,0.0,0.00,1.40,0.75,2.98,5.77,5.31,3.63,93.89,0.0,0.0,1.1.05
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1...,31878.00,40055.00,ดวง,2103.00,3578.0,4445.00,4341.00,3876.00,6195.00,3985.00,4336.00,3828.00,3368.00,0.0,0.0,2.1.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,OKR 4.2.27 จำนวนหลักสูตรดัดแปลงรถจักรยานยนต์เป...,1.00,1.00,หลักสูตร,0.00,0.0,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.0,0.0,4.2.27
297,OKR 4.2.12 จำนวนกรงตาข่ายครอบถุงขยะที่ติดตั้งถ...,2250.00,268.00,กรง (สะสม),0.00,0.0,0.00,0.00,0.00,150.00,118.00,0.00,0.00,0.00,0.0,0.0,4.2.12
298,OKR 4.2.13 ติดตั้งถังขยะ 3 ประเภทขนาด 85 ลิตร ...,5000.00,0.00,ชุด,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,4.2.13


In [ ]:
df_progress.loc[df_progress['Goal'].str.contains('OKR')].shape

(58, 16)

In [ ]:
# Contain KPI, but not Contain OKR
df_progress.loc[df_progress['Goal'].str.contains('KPI') & (df_progress['Goal'].str.contains('OKR') == False)]

,Key_Result,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
#,,,,,,,,,,,,,,,,
26,KPI 5.1.2.1 : ความสำเร็จในการแก้ไขปัญหาข้อร้อง...,90.0,17025.61,ร้อยละ,591.76,1484.03,1723.35,1918.12,2073.35,1868.41,1934.12,2483.8,1406.42,1542.25,0.0,0.0
27,KPI 5.2.1.1 : เรื่องที่ได้รับการแก้ไข ปรับปรุง...,100.0,142.00,ร้อยละ,10.00,0.00,0.00,0.00,0.00,0.00,50.00,10.0,72.00,0.00,0.0,0.0
28,KPI 5.2.1.3 : จำนวนบริการหรือกิจกรรมสาธารณะของ...,1.0,109.70,บริการต่อกิจกรรมสาธารณะ,0.00,0.00,0.00,15.00,94.70,0.00,0.00,0.0,0.00,0.00,0.0,0.0
29,KPI 5.2.2.1 : สำนักงานเขตจัดทำแผนพัฒนาเขตที่มา...,50.0,0.00,โครงการ,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0
30,KPI 5.2.2.2 : โครงการที่มาจากกระบวนการมีส่วนร่...,2017.0,163.00,ชุมชน,0.00,9.00,9.00,8.00,14.00,85.00,9.00,8.0,11.00,10.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,KPI 3.1.2.1 : กลุ่มเปราะบางที่ได้รับการจ้างงาน...,100.0,2684.33,ร้อยละ,639.33,259.00,263.00,160.50,173.50,253.50,251.83,162.0,270.67,251.00,0.0,0.0
276,KPI 3.1.2.2 : กลุ่มเปราะบางที่เข้ารับการฝึกอาช...,50.0,0.00,ร้อยละ,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0
277,KPI 3.1.2.3 : ข้าราชการ และบุคลากรของกรุงเทพมห...,100.0,300.00,ร้อยละ,0.00,0.00,0.00,0.00,0.00,0.00,100.00,100.0,100.00,0.00,0.0,0.0


In [ ]:
# Not Contain OKR & KPI

df_progress.loc[(df_progress['Goal'].str.contains('KPI') | df_progress['Goal'].str.contains('OKR')) == False]

,Key_Result,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
#,,,,,,,,,,,,,,,,
25,"อื่นๆ (ตัวชี้วัดงานประจำ) 166,748.31%",100.0,166748.31,ร้อยละ,4892.96,10198.65,9526.8,11620.32,68166.1,14361.2,11444.29,12508.83,11697.61,12331.55,0.0,0.0


In [ ]:
# # Contain OKR
df_progress.loc[df_progress['Goal'].str.contains('KPI') & (df_progress['Goal'].str.contains('OKR') == False)].shape[0] + \ ## Contain KPI, but not Contain OKR
df_progress.loc[df_progress['Goal'].str.contains('OKR')].shape[0] + \ ##
df_progress.loc[(df_progress['Goal'].str.contains('KPI') | df_progress['Goal'].str.contains('OKR')) == False].shape[0]

300

In [ ]:
df_progress

,Key_Result,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
#,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 104.01%,123.60,128.55,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.60,12.39,7.65,21.22,0.0,0.0
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 ...,200000.00,297256.00,ต้น,8734.00,24454.0,14886.00,29427.00,38176.00,51876.00,29415.00,25710.00,35950.00,38628.00,0.0,0.0
3,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 79.49%,156.00,124.00,แห่ง,2.00,6.0,7.00,16.00,13.00,22.00,11.00,9.00,14.00,24.00,0.0,0.0
4,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.38%,312.07,113.53,กม.,0.00,0.0,0.00,1.40,0.75,2.98,5.77,5.31,3.63,93.69,0.0,0.0
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1...,31878.00,39947.00,ดวง,2103.00,3578.0,4445.00,4341.00,3876.00,6195.00,3985.00,4336.00,3828.00,3260.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,OKR 4.2.27 จำนวนหลักสูตรดัดแปลงรถจักรยานยนต์เป...,1.00,1.00,หลักสูตร,0.00,0.0,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.0,0.0
297,OKR 4.2.12 จำนวนกรงตาข่ายครอบถุงขยะที่ติดตั้งถ...,2250.00,268.00,กรง (สะสม),0.00,0.0,0.00,0.00,0.00,150.00,118.00,0.00,0.00,0.00,0.0,0.0
298,OKR 4.2.13 ติดตั้งถังขยะ 3 ประเภทขนาด 85 ลิตร ...,5000.00,0.00,ชุด,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0


In [ ]:
new_col_names = {'Goal':'Goal', 'ค่าเป้าหมาย/ปี':'Yearly Goal', 'ผลดำเนินงาน (รวม)':'Acheived Goal', 'หน่วยนับ':'Unit',
       'ตค.66':'Oct 23', 'พย.66':'Nov 23', 'ธค.66':'Dec 23', 'มค.67':'Jan 24', 'กพ.67':'Feb 24', 'มี.ค.67':'Mar 24', 'เม.ย.67':'Apr 24',
       'พค.67':'May 24', 'มิ.ย.67':'Jun 24', 'กค.67':'July 24', 'สค.67':'Aug 24', 'กย.67':'Sept 24'}

In [ ]:
df_progress.rename(columns = new_col_names, inplace = True)

In [ ]:
df_progress

,Key_Result,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
#,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 99.00%,123.6,122.36,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,15.03,0.0,0.0
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 145.43%,200000.0,290856.0,ต้น,8734.0,24454.0,15086.0,29427.0,38176.0,51876.0,29470.0,25710.0,35950.0,31973.0,0.0,0.0
3,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 77.24%,156.0,120.5,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,20.5,0.0,0.0
4,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.32%,312.07,113.35,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.51,0.0,0.0
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1.20 123.20%,31878.0,39273.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,2586.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,OKR 4.2.27 จำนวนหลักสูตรดัดแปลงรถจักรยานยนต์เป็นเครื่องยนต์ไฟฟ้า 100.00%,1.0,1.0,หลักสูตร,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
297,"OKR 4.2.12 จำนวนกรงตาข่ายครอบถุงขยะที่ติดตั้งถนนสายหลัก 10,000 กรง 11.91%",2250.0,268.0,กรง (สะสม),0.0,0.0,0.0,0.0,0.0,150.0,118.0,0.0,0.0,0.0,0.0,0.0
298,OKR 4.2.13 ติดตั้งถังขยะ 3 ประเภทขนาด 85 ลิตร (Street Furniture) สำหรับคนเดินเท้าบน ทางเท้าถนนสายหลัก ถนนสายรอง 0.00%,5000.0,0.0,ชุด,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_progress.set_index(df_progress['#'], inplace = True)
df_progress.drop('#', axis = 1, inplace = True)
df_progress.head()

,KEY_RESULT,ค่าเป้าหมาย/ปี,ผลดำเนินงาน (รวม),หน่วยนับ,ตค.66,พย.66,ธค.66,มค.67,กพ.67,มี.ค.67,เม.ย.67,พค.67,มิ.ย.67,กค.67,สค.67,กย.67
#,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 99.00%,123.6,122.36,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,15.03,0.0,0.0
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 145.43%,200000.0,290856.0,ต้น,8734.0,24454.0,15086.0,29427.0,38176.0,51876.0,29470.0,25710.0,35950.0,31973.0,0.0,0.0
3,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 77.24%,156.0,120.5,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,20.5,0.0,0.0
4,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.32%,312.07,113.35,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.51,0.0,0.0
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1.20 123.20%,31878.0,39273.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,2586.0,0.0,0.0


In [ ]:
df_progress.columns

Index(['#', 'KEY_RESULT', 'ค่าเป้าหมาย/ปี', 'ผลดำเนินงาน (รวม)', 'หน่วยนับ',
       'ตค.66', 'พย.66', 'ธค.66', 'มค.67', 'กพ.67', 'มี.ค.67', 'เม.ย.67',
       'พค.67', 'มิ.ย.67', 'กค.67', 'สค.67', 'กย.67'],
      dtype='object')

In [ ]:
df_progress

,#,KEY_RESULT,ค่าเป้าหมาย/ปี,ผลดำเนินงาน (รวม),หน่วยนับ,ตค.66,พย.66,ธค.66,มค.67,กพ.67,มี.ค.67,เม.ย.67,พค.67,มิ.ย.67,กค.67,สค.67,กย.67
0,1,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 99.00%,123.6,122.36,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,15.03,0.0,0.0
1,2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 145.43%,200000.0,290856.0,ต้น,8734.0,24454.0,15086.0,29427.0,38176.0,51876.0,29470.0,25710.0,35950.0,31973.0,0.0,0.0
2,3,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 77.24%,156.0,120.5,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,20.5,0.0,0.0
3,4,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.32%,312.07,113.35,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.51,0.0,0.0
4,5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1.20 123.20%,31878.0,39273.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,2586.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,296,OKR 4.2.27 จำนวนหลักสูตรดัดแปลงรถจักรยานยนต์เป็นเครื่องยนต์ไฟฟ้า 100.00%,1.0,1.0,หลักสูตร,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
296,297,"OKR 4.2.12 จำนวนกรงตาข่ายครอบถุงขยะที่ติดตั้งถนนสายหลัก 10,000 กรง 11.91%",2250.0,268.0,กรง (สะสม),0.0,0.0,0.0,0.0,0.0,150.0,118.0,0.0,0.0,0.0,0.0,0.0
297,298,OKR 4.2.13 ติดตั้งถังขยะ 3 ประเภทขนาด 85 ลิตร (Street Furniture) สำหรับคนเดินเท้าบน ทางเท้าถนนสายหลัก ถนนสายรอง 0.00%,5000.0,0.0,ชุด,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
298,299,OKR 4.2.14 ติดตั้งถังขยะ 4 ประเภทขนาด 130 ลิตร (Street Furniture) สำหรับคนเดินเท้าบนทางเท้าถนนสายหลัก ถนนสายรอง 0.00%,10000.0,0.0,ชุด,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_progress.reset_index()

,index,#,KEY_RESULT,ค่าเป้าหมาย/ปี,ผลดำเนินงาน (รวม),หน่วยนับ,ตค.66,พย.66,ธค.66,มค.67,กพ.67,มี.ค.67,เม.ย.67,พค.67,มิ.ย.67,กค.67,สค.67,กย.67
0,0,1.0,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 99.00%,123.6,122.36,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,15.03,0.0,0.0
1,1,2.0,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 145.43%,200000.0,290856.0,ต้น,8734.0,24454.0,15086.0,29427.0,38176.0,51876.0,29470.0,25710.0,35950.0,31973.0,0.0,0.0
2,2,3.0,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 77.24%,156.0,120.5,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,20.5,0.0,0.0
3,3,4.0,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.32%,312.07,113.35,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.51,0.0,0.0
4,4,5.0,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1.20 123.20%,31878.0,39273.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,2586.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,300,296.0,OKR 4.2.27 จำนวนหลักสูตรดัดแปลงรถจักรยานยนต์เป็นเครื่องยนต์ไฟฟ้า 100.00%,1.0,1.0,หลักสูตร,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
296,301,297.0,"OKR 4.2.12 จำนวนกรงตาข่ายครอบถุงขยะที่ติดตั้งถนนสายหลัก 10,000 กรง 11.91%",2250.0,268.0,กรง (สะสม),0.0,0.0,0.0,0.0,0.0,150.0,118.0,0.0,0.0,0.0,0.0,0.0
297,302,298.0,OKR 4.2.13 ติดตั้งถังขยะ 3 ประเภทขนาด 85 ลิตร (Street Furniture) สำหรับคนเดินเท้าบน ทางเท้าถนนสายหลัก ถนนสายรอง 0.00%,5000.0,0.0,ชุด,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
298,303,299.0,OKR 4.2.14 ติดตั้งถังขยะ 4 ประเภทขนาด 130 ลิตร (Street Furniture) สำหรับคนเดินเท้าบนทางเท้าถนนสายหลัก ถนนสายรอง 0.00%,10000.0,0.0,ชุด,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_progress.head()

,#,KEY_RESULT,ค่าเป้าหมาย/ปี,ผลดำเนินงาน (รวม),หน่วยนับ,ตค.66,พย.66,ธค.66,มค.67,กพ.67,มี.ค.67,เม.ย.67,พค.67,มิ.ย.67,กค.67,สค.67,กย.67
0,1.0,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 99.00%,123.6,122.36,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,15.03,0.0,0.0
1,2.0,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 145.43%,200000.0,290856.0,ต้น,8734.0,24454.0,15086.0,29427.0,38176.0,51876.0,29470.0,25710.0,35950.0,31973.0,0.0,0.0
2,3.0,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 77.24%,156.0,120.5,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,20.5,0.0,0.0
3,4.0,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.32%,312.07,113.35,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.51,0.0,0.0
4,5.0,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1.20 123.20%,31878.0,39273.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,2586.0,0.0,0.0


In [ ]:
for i in range(1,7):
    policy_url = "https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=" + str(i) +"&per-page=50"
    print(policy_url)
    print(i)

https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=1&per-page=50
1
https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=2&per-page=50
2
https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=3&per-page=50
3
https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=4&per-page=50
4
https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=5&per-page=50
5
https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=6&per-page=50
6


In [ ]:
progress_table[0]

,#,KEY_RESULT,ค่าเป้าหมาย/ปี,ผลดำเนินงาน (รวม),หน่วยนับ,ตค.66,พย.66,ธค.66,มค.67,กพ.67,มี.ค.67,เม.ย.67,พค.67,มิ.ย.67,กค.67,สค.67,กย.67
0,1.0,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 99.00%,123.6,122.36,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,15.03,0.0,0.0
1,2.0,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 145.43%,200000.0,290856.0,ต้น,8734.0,24454.0,15086.0,29427.0,38176.0,51876.0,29470.0,25710.0,35950.0,31973.0,0.0,0.0
2,3.0,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 77.24%,156.0,120.5,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,20.5,0.0,0.0
3,4.0,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.32%,312.07,113.35,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.51,0.0,0.0
4,5.0,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1.20 123.20%,31878.0,39273.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,2586.0,0.0,0.0
5,6.0,6. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง) : OKR 1.1.03 48.62%,9536.0,4636.28,ดวง,70.0,316.0,234.0,325.0,1413.0,345.0,507.0,472.0,416.0,538.28,0.0,0.0
6,7.0,7. ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อนผัน (แห่ง) : OKR 1.1.01 116.00%,100.0,116.0,แห่ง,3.0,1.0,1.0,14.0,14.0,30.0,23.0,12.0,7.0,11.0,0.0,0.0
7,8.0,8. จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center รองรับผู้ค้าหาบเร่ (แห่ง) : OKR 7.2.05 272.73%,11.0,30.0,แห่ง,1.0,3.0,1.0,4.0,2.0,7.0,5.0,4.0,2.0,1.0,0.0,0.0
8,9.0,9. แก้ไขจุดเสี่ยงน้ำท่วม (จุด) : OKR 1.4.05 103.96%,212.0,220.4,จุด,5.0,22.0,1.0,5.0,8.01,32.0,12.2,17.2,7.45,110.54,0.0,0.0
9,10.0,10.1 แก้ปัญหาจุดจราจรฝืด (แห่ง) : OKR 1.2.05 63.77%,265.0,169.0,แห่ง,7.0,3.0,6.0,11.0,12.0,38.0,39.0,28.0,18.0,7.0,0.0,0.0


# Traffy Fondue REST API Request

In [ ]:
traffy_url = 'https://publicapi.traffy.in.th/teamchadchart-stat-api/geojson/v1?output_format=csv&name=Sukatat&org=Non&purpose=educational_project&email=skt.sukatat@gmail.com&limit=&sort=DESC'

df_traffy = pd.read_csv(traffy_url)
df_traffy.head()

,ticket_id,type,organization,organization_action,comment,coords,photo,photo_after,address,subdistrict,...,last_activity,duration_minutes_inprogress,duration_minutes_finished,duration_minutes_total,timestamp_inprogress,timestamp_finished,view_count,total_point,like,dislike
0,2024-AXHLKU,ถนน,"กรุงเทพมหานคร, เขตปทุมวัน","เขตปทุมวัน, กรุงเทพมหานคร",ปัญหาเดิมครับ อันนี้ทางเลี้ยวซ้ายผ่านตลอด แต่ม...,"100.53054,13.74574",https://storage.googleapis.com/traffy_public_b...,NaN,444 อาคาร MBK ถ. พญาไท แขวงปทุมวัน เขตปทุมวัน ...,วังใหม่,...,2024-07-25 21:53:42,NaN,NaN,NaN,NaN,NaN,0,0,0,0
1,2024-P3FQ3B,ถนน,"กรุงเทพมหานคร, เขตจตุจักร","เขตจตุจักร, กรุงเทพมหานคร",คอนโดให้จอดรถบนถนน 1 เลน ขวางการจราจร ทำให้รถต...,"100.57662,13.85703",https://storage.googleapis.com/traffy_public_b...,NaN,50/699 พหลโยธิน แขวงลาดยาว เขตจตุจักร กรุงเทพม...,ลาดยาว,...,2024-07-25 21:52:28,NaN,NaN,NaN,NaN,NaN,0,0,0,0
2,2024-LVP274,ทางเท้า,"กรุงเทพมหานคร, เขตบางกอกน้อย","เขตบางกอกน้อย, กรุงเทพมหานคร",การไฟฟ้ามาขุดทางเท้าวางสายไฟแต่ไม่เก็บงานให้เร...,"100.46957,13.75925",https://storage.googleapis.com/traffy_public_b...,NaN,453/15 ซอย จรัญสนิทวงศ์ 35 แขวงบางขุนศรี เขตบา...,บางขุนศรี,...,2024-07-25 21:51:55,NaN,NaN,NaN,NaN,NaN,0,0,0,0
3,2024-97EQYC,ถนน,"กรุงเทพมหานคร, เขตจตุจักร","เขตจตุจักร, กรุงเทพมหานคร",คอนโดให้จอดรถบนถนน 1 เลน ขวางการจราจร ทำให้รถต...,"100.57694,13.85707",https://storage.googleapis.com/traffy_public_b...,NaN,VH4G+RQ8 แขวงลาดยาว เขตจตุจักร กรุงเทพมหานคร 1...,ลาดยาว,...,2024-07-25 21:50:46,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,2024-FH9QKF,ทางเท้า,"กรุงเทพมหานคร, เขตบางซื่อ","เขตบางซื่อ, กรุงเทพมหานคร",รถยนต์ขึ้นมาจอดขวางบนทางเท้า,"100.52697,13.80626",https://storage.googleapis.com/traffy_public_b...,NaN,398/2 ถ. ประชาราษฎร์ สาย 2 แขวงบางซื่อ เขตบางซ...,บางซื่อ,...,2024-07-25 21:49:12,NaN,NaN,NaN,NaN,NaN,0,0,0,0


In [ ]:
df[:1000]

,ticket_id,type,organization,organization_action,comment,coords,photo,photo_after,address,subdistrict,...,last_activity,duration_minutes_inprogress,duration_minutes_finished,duration_minutes_total,timestamp_inprogress,timestamp_finished,view_count,total_point,like,dislike
0,ZU3CZK,อื่นๆ,"ร้องทุกข์ กทม. 1555, กรุงเทพมหานคร, เขตป้อมปรา...","เขตป้อมปราบศัตรูพ่าย, กรุงเทพมหานคร, ร้องทุกข์...",ปัญหา: ที่สำนักงานเขตป้อมปราบศัตรูพ่าย วันที่ ...,"100.51308,13.75847",https://storage.googleapis.com/traffy_public_b...,NaN,60 ถ. ศุภมิตร แขวงวัดโสมนัส เขตป้อมปราบศัตรูพ่...,วัดโสมนัส,...,2024-07-25 17:44:23,NaN,NaN,NaN,NaN,NaN,0,0,0,0
1,2024-MY4DY6,ถนน,"กรุงเทพมหานคร, เขตจตุจักร","เขตจตุจักร, กรุงเทพมหานคร",ทำท่อ ของเดิมยังขุดปิดพื้นผิวไม่เสร็จขุดเพิ่มจ...,"100.55347,13.82477",https://storage.googleapis.com/traffy_public_b...,NaN,17/6 ซอย วิภาวดีรังสิต 17 แยก 10 แขวงจตุจักร เ...,จตุจักร,...,2024-07-25 17:43:29,NaN,NaN,NaN,NaN,NaN,0,0,0,0
2,2024-83XNTW,ถนน,"กรุงเทพมหานคร, เขตบางนา","เขตบางนา, กรุงเทพมหานคร",ฝาท่อน้ำกลางถนนไม่เรียบทั้งช่วงถนน ทำให้ช่วงล่...,"100.64481,13.66191",https://storage.googleapis.com/traffy_public_b...,NaN,1 ศรีนครินทร์ 26 บางนาใต้ เขตบางนา กรุงเทพมหาน...,บางนาเหนือ,...,2024-07-25 17:42:24,NaN,NaN,NaN,NaN,NaN,0,0,0,0
3,2024-KTPQUF,อื่นๆ,"กรุงเทพมหานคร, เขตบางแค","เขตบางแค, กรุงเทพมหานคร",เอาป้ายร้านค้า “แอร์“ มาติดบนถนนสาธารณะ มีการก...,"100.37594,13.71474",https://storage.googleapis.com/traffy_public_b...,NaN,86 ซอย เศรษฐกิจ 27 แขวงบางแคเหนือ เขตบางแค กรุ...,บางแคเหนือ,...,2024-07-25 17:38:57,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,2024-KUKFDP,"อื่นๆ,วินมอไซค์","เพื่อนชัชชาติ, เขตบางกอกน้อย, กรุงเทพมหานคร","กรุงเทพมหานคร, เขตบางกอกน้อย, เพื่อนชัชชาติ",วินมอเตอร์ไซค์เก็บเกินราคา ป้าย ผู้โดยสารนั่งห...,"100.47325,13.76282",https://storage.googleapis.com/traffy_public_b...,NaN,QF7F+696 แขวงศิริราช เขตบางกอกน้อย กรุงเทพมหาน...,ศิริราช,...,2024-07-25 17:38:55,NaN,NaN,NaN,NaN,NaN,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,HBJJXK,เสียง,"ร้องทุกข์ กทม. 1555, กรุงเทพมหานคร, เขตบางรัก,...","ฝ่ายสิ่งแวดล้อมฯ เขตบางรัก, เขตบางรัก, กรุงเทพ...",ศูนย์เรื่องราวร้องทุกข์ ได้รับการประสานผ่านระบ...,"100.52348,13.73067",https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,สำนักงานเขตบางรัก ถนน นเรศ แขวงสี่พระยา เขตบาง...,สี่พระยา,...,2024-07-25 14:08:29,420.0,1046.0,1046.0,2024-07-24 22:56:39,2024-07-25 09:22:39,0,0,0,0
996,2024-6K6RNY,ถนน,"กรุงเทพมหานคร, เขตบางกอกน้อย, สน.บางกอกน้อย, ก...","เขตบางกอกน้อย, กองบังคับการตำรวจนครบาล 7 (บก.น...",รถติดมากครับ สี่แยกศิริราช,"100.48329,13.75741",https://storage.googleapis.com/traffy_public_b...,NaN,รพ.ศิริราช แขวงศิริราช เขตบางกอกน้อย กรุงเทพมห...,ศิริราช,...,2024-07-24 16:05:48,NaN,NaN,NaN,NaN,NaN,0,0,0,0
997,2024-FNXKXB,ผิดกฎจราจร,"กรุงเทพมหานคร, เขตบางกอกน้อย, สน.บางกอกน้อย, ก...","เขตบางกอกน้อย, กองบังคับการตำรวจนครบาล 7 (บก.น...",มีรถสองแถวสีแดงทะเบียน 15-4618 กรุงเทพมหานคร ว...,"100.48185,13.75972",https://storage.googleapis.com/traffy_public_b...,NaN,166/1 ถนน วัดสุทธาวาส แขวงศิริราช เขตบางกอกน้อ...,ศิริราช,...,2024-07-24 16:03:25,NaN,NaN,NaN,NaN,NaN,0,0,0,0
998,2024-NP3DBL,ไฟฟ้า,"กรุงเทพมหานคร, เขตคลองเตย, ฝ่ายโยธา เขตคลองเตย...","ฝ่ายโยธา เขตคลองเตย, เขตคลองเตย, การไฟฟ้านครหล...",เรื่องเพิ่มแสงสว่างถนน ในบริเวณซอยสุขุมวิท 40 ...,"100.58056,13.72166",https://storage.googleapis.com/traffy_public_b...,NaN,26 ซ. เหมวงศ์ แขวง พระโขนง เขตคลองเตย กรุงเทพม...,พระโขนง,...,2024-07-24 16:29:07,30.0,NaN,30.0,2024-07-24 16:25:15,NaN,0,0,0,0
